## Sentiment Analysis_Helperfunctions

### Load the all packages

In [8]:
from __future__ import print_function

%matplotlib inline

import tensorflow
import keras
import nltk
import os
import re
import itertools
import wordcloud
import csv
import testfixtures
import statsmodels
import locale
import glob
import os.path
import requests
import tarfile
import sys
import codecs
import smart_open

import numpy as np
import _pickle as pickle
import pandas as pd

from IPython.display import SVG
from IPython.display import Image

from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding, SpatialDropout1D
from keras.layers import LSTM
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Flatten
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from keras.utils.data_utils import get_file
from imp import reload
from PIL import Image
from itertools import chain
from matplotlib import pyplot as plt
from numpy.random import normal
from wordcloud import WordCloud, STOPWORDS

___
### Save the dataset to a CSV file

In [2]:
# save dataset to .csv

# index/word mapping
idx = imdb.get_word_index()
idx2word = {v: k for k, v in idx.items()}

# get full dataset copied from Keras and split into train/test and separate features from labels
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')

# split 
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

# move train data to csv
with open('train.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    for i in range(0, len(x_train)):
        label = labels_train[i]
        review = ' '.join([idx2word[o] for o in x_train[i]])
        writer.writerow([review, label])
        
# move test data to csv
with open('test.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    for i in range(0, len(x_test)):
        label = labels_test[i]
        review = ' '.join([idx2word[o] for o in x_test[i]])
        writer.writerow([review, label])

In [3]:
train_data = pd.read_csv('train.csv', header=None)
test_data = pd.read_csv('test.csv', header=None)

### Download the IMDB dataset and save the the text files into one big text file

In [9]:
dirname = 'aclImdb'
filename = 'aclImdb_v1.tar.gz'
locale.setlocale(locale.LC_ALL, 'C')

if sys.version > '3':
    control_chars = [chr(0x85)]
else:
    control_chars = [unichr(0x85)]

# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')
    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')
    return norm_text

import time
start = time.clock()

if not os.path.isfile('aclImdb/alldata-id.txt'):
    if not os.path.isdir(dirname):
        if not os.path.isfile(filename):
            # Download IMDB archive
            print("Downloading IMDB archive...")
            url = u'http://ai.stanford.edu/~amaas/data/sentiment/' + filename
            r = requests.get(url)
            with open(filename, 'wb') as f:
                f.write(r.content)
        tar = tarfile.open(filename, mode='r')
        tar.extractall()
        tar.close()

    # Concatenate and normalize test/train data
    print("Cleaning up dataset...")
    folders = ['train/pos', 'train/neg', 'test/pos', 'test/neg', 'train/unsup']
    alldata = u''
    for fol in folders:
        temp = u''
        output = fol.replace('/', '-') + '.txt'
        # Is there a better pattern to use?
        txt_files = glob.glob(os.path.join(dirname, fol, '*.txt'))
        for txt in txt_files:
            with smart_open.smart_open(txt, "rb") as t:
                t_clean = t.read().decode("utf-8")
                for c in control_chars:
                    t_clean = t_clean.replace(c, ' ')
                temp += t_clean
            temp += "\n"
        temp_norm = normalize_text(temp)
        with smart_open.smart_open(os.path.join(dirname, output), "wb") as n:
            n.write(temp_norm.encode("utf-8"))
        alldata += temp_norm

    with smart_open.smart_open(os.path.join(dirname, 'alldata-id.txt'), 'wb') as f:
        for idx, line in enumerate(alldata.splitlines()):
            num_line = u"_*{0} {1}\n".format(idx, line)
            f.write(num_line.encode("utf-8"))

end = time.clock()
print ("Total running time: ", end-start)

Total running time:  0.0005919999999992598


In [10]:
import io
import pandas as pd
import numpy as np

with io.open('aclImdb/train-pos.txt', encoding='utf-8') as f:
    train_pos = pd.DataFrame({'review': list(f)})    
with io.open('aclImdb/train-neg.txt', encoding='utf-8') as f:
    train_neg = pd.DataFrame({'review': list(f)}) 
train_reviews = pd.concat([train_neg, train_pos], ignore_index=True)

with io.open('aclImdb/test-pos.txt', encoding='utf-8') as f:
    test_pos = pd.DataFrame({'review': list(f)})
with io.open('aclImdb/test-neg.txt', encoding='utf-8') as f:
    test_neg = pd.DataFrame({'review': list(f)})    
test_reviews = pd.concat([test_neg, test_pos], ignore_index=True)
  
X_train = train_reviews['review']
X_test = test_reviews['review']

y_train = np.append(np.zeros(12500), np.ones(12500))
y_test = np.append(np.zeros(12500), np.ones(12500))

In [11]:
# print a review from the original dataset
print("Review-Example from the original dataset:")
csvfile = open('/Users/ninosyonan/Documents/Uni/Master/3. Semester/Information Systems Seminar/Python/train.csv','r')
csvFileArray = []
for row in csv.reader(csvfile, delimiter = '.'):
    csvFileArray.append(row)
print(csvFileArray[0])

Review-Example from the original dataset:


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 7123: ordinal not in range(128)

#### We use a logistic regression to classify which features are important

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

stopwords_nltk = set(stopwords.words("english"))
relevant_words = set(['not', 'nor', 'no', 'wasn', 'ain', 'aren', 'very', 'only', 'but', 'don', 'isn', 'weren'])
stopwords_filtered = list(stopwords_nltk.difference(relevant_words))
vectorizer = CountVectorizer(stop_words =  stopwords_filtered, max_features = 10000, ngram_range = (1,2))
X_train_features = vectorizer.fit_transform(X_train)
X_test_features = vectorizer.transform(X_test)

logistic_model = LogisticRegression(C=0.03) 
logistic_model.fit(X_train_features, y_train)